In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 15.4 MB/s eta 0:00:00


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor

In [ ]:
import torch
from torch.nn import functional as F
from IPython.display import display
from PIL import Image
import numpy as np
import requests
import matplotlib.pyplot as plt
from io import BytesIO

In [ ]:
from qwen_vl_utils import process_vision_info

In [ ]:
cat='/content/drive/My Drive/Clip tokenization dataset/cat.jpg'
img=Image.open(cat)

In [ ]:
model=Qwen2_5_VLForConditionalGeneration.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct",device_map="auto",torch_dtype="auto")
processor=AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

In [ ]:
msgs = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": (
                    "You are an object detector. The format of your output should be a valid JSON object "
                    "{'bbox_2d': [x1, y1, x2, y2], 'label': 'class'} where class is the name of the class you are detecting."
                )
            }
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": img},
            {
                "type": "text",
                "text": "Outline the position of elephants"
            }
        ],
    }
]

In [ ]:
text_prompt=processor.apply_chat_template(msgs,tokenize=False,add_generation_prompt=True)
img_inputs,video_outputs=process_vision_info(msgs)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
inputs=processor(
    text=text_prompt,
    images=img_inputs,
    return_tensors="pt",
    padding=True,
).to(device)

In [ ]:
with torch.no_grad():
  output=model.generate(**inputs,max_new_tokens=1024)
print(output.shape)

torch.Size([1, 175])


In [ ]:
output = processor.batch_decode(
      output[:, inputs.input_ids.shape[-1]:],
      skip_special_tokens=False
  )[0]
print(output)

```json
[
	{"bbox_2d": [0, 13, 252, 174], "label": "elephants"}
]
```<|im_end|>
